In [1]:
import numpy as np
import random
from tqdm import tqdm

# ファイルを読み込む
file1 = '../data/processed/melspec_20250511_1415/birdclef2025_melspec_5sec_256_256.npy'
file2 = '../data/processed/melspec_maxRMS/birdclef2025_melspec_5sec_256_256.npy'

dict1 = np.load(file1, allow_pickle=True).item()
dict2 = np.load(file2, allow_pickle=True).item()

# キーリストを取得
keys1 = set(dict1.keys())
keys2 = set(dict2.keys())

# キーセットの比較
if keys1 != keys2:
    print("キーセットが一致しません。")
else:
    print("キーセットは一致しています。")

    keys = list(keys1)
    num_keys_to_check = 10  # チェックするキーの数
    num_elements_to_check_per_key = 1  # 各キー内でチェックする要素の数

    if len(keys) <= num_keys_to_check:
        sampled_keys = keys
    else:
        sampled_keys = random.sample(keys, num_keys_to_check)

    all_match = True

    for key in tqdm(sampled_keys, desc="Checking sampled keys"):
        arr1 = dict1[key]
        arr2 = dict2[key]

        if arr1.shape != arr2.shape:
            print(f"キー {key} の shape が違います: {arr1.shape} vs {arr2.shape}")
            all_match = False
            break

        flat1 = arr1.flatten()
        flat2 = arr2.flatten()

        if len(flat1) <= num_elements_to_check_per_key:
            indices = range(len(flat1))
        else:
            indices = random.sample(range(len(flat1)), num_elements_to_check_per_key)

        for idx in indices:
            if not np.isclose(flat1[idx], flat2[idx], atol=1e-5):
                print(f"キー {key} の index {idx} で値が違います: {flat1[idx]} vs {flat2[idx]}")
                all_match = False
                break

        if not all_match:
            break

    if all_match:
        print("一部サンプリング比較では一致しました！")
    else:
        print("一部サンプリング比較で不一致が見つかりました。")

キーセットは一致しています。


Checking sampled keys:   0%|          | 0/10 [00:00<?, ?it/s]

キー greegr-XC764651 の index 31747 で値が違います: 0.34115123748779297 vs 0.34604448080062866
一部サンプリング比較で不一致が見つかりました。
